In [2]:
import numpy as np

In [35]:
MAX_CAP = 10
# values = [10, 9, 5, 1, 7, 2, 7, 2, 6, 2, 7, 4, 4, 1, 2, 20]
values = np.random.randint(low=0, high=MAX_CAP, size=(20))
# values.sort(reverse=True)
# print(values)

def getBinPackingResults(values:list, MAX_CAP:float) -> tuple[list, list]:

    """
    Finds a near-optimal solution to the bin packing problem by minimizing the total number of bins a set of values will fit into 
    assuming each bin has an identical max capacity.
    
    This can be used to optimally schedule batches of network training to minimize the number of individual batches while maximizing parallelism.
    
    Arguments:
        values: A list of values which represents the estimated memory in MB each model will use during training
        MAX_CAP: A float representing the total MB capacity of the GPU
        
    Returns:
        (valueBins, indexBins)
        valueBins: A list of lists where each sublist represents the values of items that sum to less than MAX_CAP
        indexBins: A list of lists where each sublist holds the indices of the original values which should be grouped to minimize bin counts.
    """

    values = np.array(values, dtype=np.float32)
    
    sortedIndices = np.argsort(values, kind='stable')[::-1]
    values = values[sortedIndices]
    
    # Initialize empty bins and indices since we know at most, there can be N bins for N values
    bins = [[] for i in range(len(values))]
    binIndices = [[] for i in range(len(values))]
    binCapacities = [int(MAX_CAP - np.sum(bin)) for bin in bins]
    remainingCapacities = (np.ones((len(bins))) * MAX_CAP)
    
    for idx in range(len(values)):
        
        currentValue = values[idx]
        currentValueIdx = sortedIndices[idx]

        for bCapIdx, binCapacity in enumerate(binCapacities):
            
            remainingCapacity = binCapacity - currentValue
            remainingCapacities[bCapIdx] = remainingCapacity

        # If a value is out of the maximum range, add it to its own list
        if currentValue > MAX_CAP:
            bins.append([currentValue])
            binIndices.append([currentValueIdx])
            continue

        bestBinIdx = np.where(remainingCapacities >= 0)[0][np.argmin(remainingCapacities[remainingCapacities >= 0])]
        
        bins[bestBinIdx].append(currentValue)
        binIndices[bestBinIdx].append(currentValueIdx)
        binCapacities[bestBinIdx] = np.sum(binCapacities[bestBinIdx]) - currentValue

    return [bin for bin in bins if len(bin)>0], [idx for idx in binIndices if len(idx)>0]
            
        
        
print(values)
print([i for i in range(len(values))])
print()
groupings, indices = getBinPackingResults(values, MAX_CAP)
print(groupings)
print(indices)

[0 5 2 9 4 8 3 8 4 2 1 6 3 7 5 6 2 2 9 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

[[9.0, 1.0, 0.0], [9.0], [8.0, 2.0], [8.0, 2.0], [7.0, 3.0], [7.0, 3.0], [6.0, 4.0], [6.0, 4.0], [5.0, 5.0], [2.0, 2.0]]
[[18, 10, 0], [3], [7, 17], [5, 16], [19, 12], [13, 6], [15, 8], [11, 4], [14, 1], [9, 2]]
